In [1]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class  # Mirror the matrix
    
    # Print the delta class matrix
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    
    # Calculate the distance matrix for delta class
    distance_matrix_class = np.linalg.norm(delta_class_matrix[:, np.newaxis] - delta_class_matrix, axis=2)
    
    # Calculate and print average distances for delta class
    average_distances_class = np.mean(distance_matrix_class, axis=1)
    print(f"Round {round_num} Average distances (Delta Class):")
    print(average_distances_class)
    
    # Identify the client with the highest average distance for delta class
    max_distance_client_class = np.argmax(average_distances_class)
    print(f"Round {round_num}, Client with highest average distance (Delta Class): {max_distance_client_class}")
    
    # Calculate the delta score matrix
    delta_score_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, Δ_score, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_score_matrix[i, j] = Δ_score
            delta_score_matrix[j, i] = Δ_score  # Mirror the matrix
    
    # Print the delta score matrix
    print(f"Round {round_num} Delta Score Matrix:")
    print(delta_score_matrix)
    
    # Calculate the distance matrix for delta score
    distance_matrix_score = np.linalg.norm(delta_score_matrix[:, np.newaxis] - delta_score_matrix, axis=2)
    
    # Calculate and print average distances for delta score
    average_distances_score = np.mean(distance_matrix_score, axis=1)
    print(f"Round {round_num} Average distances (Delta Score):")
    print(average_distances_score)
    
    # Identify the client with the highest average distance for delta score
    max_distance_client_score = np.argmax(average_distances_score)
    print(f"Round {round_num}, Client with highest average distance (Delta Score): {max_distance_client_score}")
    
    # Calculate the P_KS matrix
    p_ks_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, p_ks, _ = perform_differential_testing(predictions[i], predictions[j])
            p_ks_matrix[i, j] = p_ks
            p_ks_matrix[j, i] = p_ks  # Mirror the matrix
    
    # Print the P_KS matrix
    print(f"Round {round_num} P_KS Matrix:")
    print(p_ks_matrix)
    
    # Calculate the distance matrix for P_KS
    distance_matrix_ks = np.linalg.norm(p_ks_matrix[:, np.newaxis] - p_ks_matrix, axis=2)
    
    # Calculate and print average distances for P_KS
    average_distances_ks = np.mean(distance_matrix_ks, axis=1)
    print(f"Round {round_num} Average distances (P_KS):")
    print(average_distances_ks)
    
    # Identify the client with the highest average distance for P_KS
    max_distance_client_ks = np.argmax(average_distances_ks)
    print(f"Round {round_num}, Client with highest average distance (P_KS): {max_distance_client_ks}")
    
        # Calculate the P_X2 matrix
    p_x2_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, _, p_x2 = perform_differential_testing(predictions[i], predictions[j])
            p_x2_matrix[i, j] = p_x2
            p_x2_matrix[j, i] = p_x2  # Mirror the matrix
    
    # Print the P_X2 matrix
    print(f"Round {round_num} P_X2 Matrix:")
    print(p_x2_matrix)
    
    # Calculate the distance matrix for P_X2
    distance_matrix_x2 = np.linalg.norm(p_x2_matrix[:, np.newaxis] - p_x2_matrix, axis=2)
    
    # Calculate and print average distances for P_X2
    average_distances_x2 = np.mean(distance_matrix_x2, axis=1)
    print(f"Round {round_num} Average distances (P_X2):")
    print(average_distances_x2)
    
    # Identify the client with the highest average distance for P_X2
    max_distance_client_x2 = np.argmax(average_distances_x2)
    print(f"Round {round_num}, Client with highest average distance (P_X2): {max_distance_client_x2}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.8427167), ('loss', 0.5798236)]))])
Round 1 Delta Class Matrix:
[[   0. 8554. 8681. 8806. 8851. 8581. 8822. 8592. 9027. 8471.]
 [8554.    0. 2138. 2019. 2127. 1998. 2184. 1896. 2117. 2045.]
 [8681. 2138.    0. 2091. 2061. 2123. 2233. 2005. 2150. 2098.]
 [8806. 2019. 2091.    0. 2112. 1929. 1970. 1733. 1932. 1978.]
 [8851. 2127. 2061. 2112.    0. 2007. 2170. 1974. 2022. 2158.]
 [8581. 1998. 2123. 1929. 2007.    0. 2048. 1800. 2056. 1954.]
 [8822. 2184. 2233. 1970. 2170. 2048.    0. 1912. 2121. 2109.]
 [8592. 1896. 2005. 1733. 1974. 1800. 1912.    0. 1891. 1819.]
 [9027. 2117. 2150. 1932. 2022. 2056. 2121. 1891.    0. 2020.]
 [8471. 2045. 2098. 1978. 2158. 1954. 2109. 1819. 2020.    0.]]
Round 1 Average distances (Delta Class):
[20308.42523839  4596.64370282  4650.78621507  4526.59340619
  4628.151